In [ ]:
import pandas as pd
import glob
import pyodbc as py
from config import configscape

In [ ]:
def loadreservationsql(data,query):
    
    Driver          = configscape.Driver
    Server_name     = configscape.Server_name
    Database_name   = configscape.Database_name

    conn_string = f""" 
    Driver={{{Driver}}};
    Server={Server_name};
    Database={Database_name};
    Trusted_Connection=yes;
    """
    
    con = py.connect(conn_string)
    cursor = con.cursor() 
    cursor.fast_executemany = True
    
    inputsize =configscape.inputsize
    cursor.setinputsizes(inputsize)

    try:
        print('Inserting Reservation...')
        result = cursor.executemany(query,data)
        cursor.commit()       
    except Exception as er:
        print('An Error Occur')
        print(er)
        cursor.close()
        con.close()
        print('Closing SQL Connection')
        return 'Error'


In [ ]:
def combinefile():
    list_output = glob.glob(r'Output/*.csv')
    combine_list = pd.DataFrame()
    for file in list_output:
        combine_list =combine_list.append(pd.read_csv(file,sep=','))
    strlist = combine_list.dtypes[combine_list.dtypes =='object'].index
    combine_list = combine_list.drop_duplicates()
    combine_list[strlist] =combine_list[strlist].astype('string') 
    combine_list['dest_ufi'] =combine_list['dest_ufi'].astype('float') 
    combine_list['inuse'] = 0
    combine_list.columns = configscape.col_name
    combine_list = combine_list[configscape.order_col].fillna(0)

    return combine_list

In [ ]:
file_combine= combinefile()
# file_combine.to_csv('Combine_task.csv')
loadreservationsql(file_combine.values.tolist(),configscape.query)